In [13]:
import datetime
import pandas as pd
import numpy as np
import random as rand

#테스트 파일 로딩
coin_name = "KRW-XRP"
df_org = pd.read_csv("./data/{}-index.csv".format(coin_name))
print(df_org.shape)
print(df_org.columns)

(199878, 31)
Index(['t', 'o', 'h', 'l', 'c', 'v', 'sma7', 'sma14', 'sma99', 'wma7', 'wma25',
       'wma99', 'ema7', 'ema25', 'ema99', 'macd', 'macd_s', 'macd_d', 'rsi7',
       'rsi14', 'rsi28', 'srsi', 'srsik', 'srsid', 'bh', 'bhi', 'bl', 'bli',
       'bm', 'bw', 'vwap'],
      dtype='object')


In [20]:
#단계별 구매 수량
def get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate):
    buy_amt = 0
    buy_amt_list = [0.0]
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_amt = round(buy_amt + temp_amt, 4)
        buy_amt_list.append(buy_amt)
    return buy_amt_list

#손실 최소화 실현 금액 계산
def get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate):
    buy_amt = 0
    buy_price = 0
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_price = round(buy_price + close * temp_amt, 4)
        buy_amt = round(buy_amt + temp_amt, 4)
    return round(buy_price * max_loss_rate, 4)

#수익률 보정: 총매수금액 200 coin(xrp) 기준으로 조정
def adj_revenue(revenue, close, buy_amt_unit, buy_cnt_limit, increace_rate):
    open_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
    max_amt = open_amt_list[len(open_amt_list)-1]
    adj_revenue = (200 * revenue) / max_amt
    return adj_revenue


def run_test(config):
    revenue_rate  = config['revenue_rate']
    max_loss_rate = config['max_loss_rate']
    increace_rate = config['increace_rate']
    buy_cnt_limit = int(config['buy_cnt_limit'])
    buy_amt_unit  = config['buy_amt_unit']
    trade_fee = 0.001 #거래수수료
    close = 1300
    buy_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
    max_loss = get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate)

    buy_cnt = 0 
    buy_price = 0
    buy_amt = 0
    revenue = 0
    revenue_t = 0

    df = df_org.iloc[df_org.shape[0]-144000:df_org.shape[0]-124000,]
    # df = df_org2
    for i in range(0, df.shape[0]-1):
        close1 = round(df.iloc[i:i+1,]['c'].values[0],4)
        close2 = round(df.iloc[i+1:i+2,]['c'].values[0],4) #다음 분의 종가로 산다
        wma7 = round(df.iloc[i:i+1,]['wma7'].values[0],4)
        wma99 = round(df.iloc[i:i+1,]['wma99'].values[0],4)
        vwap = round(df.iloc[i:i+1,]['vwap'].values[0],4)
        macd_s = round(df.iloc[i:i+1,]['macd_s'].values[0],4)
        macd_d = round(df.iloc[i:i+1,]['macd_d'].values[0],4)

        #stop loss
        loss = buy_price - close2*buy_amt
        if loss > max_loss:
            revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
            revenue = round(revenue + revenue_t,4)
            buy_cnt = 0
            buy_amt = 0
            buy_price = 0
            continue

        #take profit
        tp_revenue = close2*buy_amt - (buy_price + buy_price*revenue_rate)
        if buy_cnt > 0 and tp_revenue > 0:
            revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
            revenue = round(revenue + revenue_t,4)
            buy_cnt = 0
            buy_amt = 0
            buy_price = 0
            continue

        #buy
        if buy_cnt < buy_cnt_limit and close2 < vwap and close2 < wma7 and wma7 > wma99:
            temp_amt = buy_amt_unit + buy_amt*increace_rate
            buy_price = round(buy_price + close2*temp_amt,4)
            buy_amt = round(buy_amt + temp_amt,4)
            buy_cnt = buy_cnt + 1 
        
    return adj_revenue(revenue, close, buy_amt_unit, buy_cnt_limit, increace_rate)

In [21]:
def random_select():
    config_data = {
        'revenue_rate':rand.uniform(0.005,0.015),
        'max_loss_rate':rand.uniform(0.05,0.3),
        'increace_rate':rand.uniform(0.1,0.4),
        'buy_cnt_limit':rand.uniform(5,20),
        'buy_amt_unit':rand.uniform(5,40)
    }
    return config_data

results = []

for i in range(50):
    config_data = random_select()
    revenue = run_test(config_data)
    print("*config:{} result:{}".format(config_data, revenue))
    result = []
    result.append(config_data)
    result.append(revenue)
    results.append(result)


*config:{'revenue_rate': 0.0073643011434717034, 'max_loss_rate': 0.2078220155400649, 'increace_rate': 0.11936388952775001, 'buy_cnt_limit': 18.711411765802087, 'buy_amt_unit': 25.982108062592133} result:-128149.37015328482
*config:{'revenue_rate': 0.008552287171405309, 'max_loss_rate': 0.17144158309442142, 'increace_rate': 0.31993139393876924, 'buy_cnt_limit': 9.55449228880118, 'buy_amt_unit': 16.36429667712276} result:-158380.71723372975
*config:{'revenue_rate': 0.012417667222756619, 'max_loss_rate': 0.1407176025280103, 'increace_rate': 0.22586977152646312, 'buy_cnt_limit': 6.98879427577679, 'buy_amt_unit': 5.301800521631165} result:-163085.47878986355
*config:{'revenue_rate': 0.014154319447728207, 'max_loss_rate': 0.2358696131358647, 'increace_rate': 0.32631612056727954, 'buy_cnt_limit': 8.465739777167173, 'buy_amt_unit': 22.829746532331075} result:-148573.52753915952
*config:{'revenue_rate': 0.012779780282068342, 'max_loss_rate': 0.20822829918781072, 'increace_rate': 0.2725255430782

*config:{'revenue_rate': 0.010563606877321136, 'max_loss_rate': 0.06971484012361896, 'increace_rate': 0.13102827488077395, 'buy_cnt_limit': 5.286303353470733, 'buy_amt_unit': 29.388090211779613} result:-154514.1058103463
*config:{'revenue_rate': 0.013522099691791686, 'max_loss_rate': 0.25276129753229926, 'increace_rate': 0.33704643252098543, 'buy_cnt_limit': 12.908072382058968, 'buy_amt_unit': 26.398538263033345} result:-159260.15551436332
*config:{'revenue_rate': 0.0053504784491969345, 'max_loss_rate': 0.10025318642528798, 'increace_rate': 0.35409957606484666, 'buy_cnt_limit': 5.712100407734192, 'buy_amt_unit': 5.548692339980738} result:-64404.00268018976
*config:{'revenue_rate': 0.007837538126324652, 'max_loss_rate': 0.2691309491017102, 'increace_rate': 0.1548487700541786, 'buy_cnt_limit': 15.775438143977984, 'buy_amt_unit': 29.93278430169802} result:-137625.89932280648
*config:{'revenue_rate': 0.005381114063044139, 'max_loss_rate': 0.10719943908780284, 'increace_rate': 0.26820138676

In [23]:
results.sort(key=lambda x:x[1], reverse=True) #2차원 배열 정렬하기
results

[[{'revenue_rate': 0.005212356237250213,
   'max_loss_rate': 0.1128899057559669,
   'increace_rate': 0.1465560586177363,
   'buy_cnt_limit': 15.156110657924385,
   'buy_amt_unit': 24.616118537520844},
  -13233.967336056516],
 [{'revenue_rate': 0.008055182379294833,
   'max_loss_rate': 0.09986355832205092,
   'increace_rate': 0.22955532132930423,
   'buy_cnt_limit': 14.004614175812838,
   'buy_amt_unit': 18.759124777424848},
  -23249.790149602155],
 [{'revenue_rate': 0.013430117291232519,
   'max_loss_rate': 0.07660373217990131,
   'increace_rate': 0.29312930951140637,
   'buy_cnt_limit': 19.803066279815106,
   'buy_amt_unit': 17.75420007069027},
  -23726.037317879436],
 [{'revenue_rate': 0.00881727569094047,
   'max_loss_rate': 0.1505386662933122,
   'increace_rate': 0.21464783347684013,
   'buy_cnt_limit': 16.161625851641656,
   'buy_amt_unit': 20.06208726071896},
  -28054.582847016194],
 [{'revenue_rate': 0.007158497012070378,
   'max_loss_rate': 0.07804239521838789,
   'increace_rat